In [1]:
from random import randint
para={'XGBClassifier':{'learning_rate':[0.1,0.01,0.001],
                      'n_estimators':[600,800,1000],
                      'max_depth':[4,5,6],
                      'min_child_weight':[4,5,6],
                      'gamma':[i/10.0 for i in range(0,4)]},
      'Random Forest Classifier':{'n_estimators':[600,800,1000],
                                 'max_depth':[4,5,6],
                                 'min_samples_split':[],
                                 'max_leaf_nodes':[],
                                 'max_features' : ['sqrt', 'log2']},
      'Decision Tree':{'criterion':['gini'],
                       'splitter':[2,4,6],
                       'max_depth':[],
                       'min_samples_leaf':[],
                       'max_features' :['sqrt', 'log2']},
      'CatboostClassifier':{'iterations':[],
                           'learning_rate':[],
                           'rsm':[],
                           'loss_function':[],
                           'max_depth':randint(3, 10)},
      'LogisticRegression':{'max_iter':[],
                           'solver':['lbfgs','sag','saga','liblinear'],
                           'penalty':['None','l2'],
                           'c_values':[100,10,1,0.1,0.01]                           },
      'KNeighborsClassifier':{'n_neighbors' : range(1, 21, 2),
                              'weights' : ['uniform', 'distance'],
                              'metric' : ['euclidean', 'manhattan', 'minkowski']},
      'SVMClassifier':{'kernel' : ['poly', 'rbf', 'sigmoid'],
                      'C' : [50, 10, 1.0, 0.1, 0.01],
                      'gamma' :['scale']},
      'GradientBoostingClassifier':{'n_estimators' : [10, 100, 1000],
                                    'learning_rate' : [0.001, 0.01, 0.1],
                                    'subsample' : [0.5, 0.7, 1.0],
                                    'max_depth' : [3, 7, 9]},
      'AdaBoost Classifier':{'n_estimators': [2, 3, 4],
                             'learning_rate': [(0.97 + x / 100) for x in range(0, 4)],
                             'algorithm': ['SAMME', 'SAMME.R']}
     }

In [2]:
list(para.values())[1]

{'n_estimators': [600, 800, 1000],
 'max_depth': [4, 5, 6],
 'min_samples_split': [],
 'max_leaf_nodes': [],
 'max_features': ['sqrt', 'log2']}

In [3]:
list(para.keys())[1]

'Random Forest Classifier'

In [4]:
import numpy as np
np.linspace(0,0.2,5)

array([0.  , 0.05, 0.1 , 0.15, 0.2 ])

In [5]:
para['Random Forest Classifier']

{'n_estimators': [600, 800, 1000],
 'max_depth': [4, 5, 6],
 'min_samples_split': [],
 'max_leaf_nodes': [],
 'max_features': ['sqrt', 'log2']}